In [157]:
import pandas as pd
import xmltodict
import json
import re, string
import numpy as np

In [92]:
df_click = pd.read_csv('Datasets/clickstream-enwiki-2020-01.tsv', delimiter='\t', encoding='utf-8', names=['referer', 'resource', 'path', 'count'])

In [93]:
#df_click.head(10)

In [94]:
# with open('Datasets/enwiki-20200101-pages-articles-multistream-index1.txt-p10p30302', encoding='utf8') as file:
    data_index = file.read()

In [95]:
with open('Datasets/enwiki-20200101-pages-articles-multistream1.xml-p10p30302', encoding='utf8') as file:
    #data_text = file.read()
    doc = xmltodict.parse(file.read())

In [96]:
#print(data_index[:300])

In [97]:
#print(data_text[:1000000])

In [98]:
print(doc['mediawiki']['page'][11])

OrderedDict([('title', 'Autism'), ('ns', '0'), ('id', '25'), ('revision', OrderedDict([('id', '930524291'), ('parentid', '930523037'), ('timestamp', '2019-12-13T02:28:01Z'), ('contributor', OrderedDict([('username', 'AnomieBOT'), ('id', '7611264')])), ('comment', 'Fixing reference errors'), ('model', 'wikitext'), ('format', 'text/x-wiki'), ('text', OrderedDict([('@xml:space', 'preserve'), ('#text', '{{about|the classic autistic disorder|other conditions sometimes called "autism"|Autism spectrum|the journal|Autism (journal)}}\n{{pp-semi-indef}}\n{{pp-move-indef}}\n{{short description|Neurodevelopmental disorder involving problems with social interaction and communication, and the presence of repetitive behaviors}}\n{{bots|deny=Monkbot}} <!-- keep Monkbot from visiting this page -->\n{{Use dmy dates|date=August 2018}}\n{{Use American English|date=August 2016}}\n{{Infobox medical condition (new)\n| name            = Autism\n| image           = Autism-stacking-cans 2nd edit.jpg\n| alt     

In [128]:
df_text = pd.DataFrame(columns=['title', 'text', 'wiki_link', 'redirect'])

for page in doc['mediawiki']['page']:
    title = page['title']
    text_w = ''
    text_s = ''
    wiki_link = ''
    redirect = 'F'
    
    if 'redirect' in page:
        # only keeping redirecting link
        txt = re.search('(\[\[(.*?)\]\])', page['revision']['text']['#text']).group(1)
        txt = re.sub('\[*\]*', '', txt)
        redirect = 'T'
        wiki_link = txt.strip()
        
    else:
        # getting rid of {{~}}, [[File:~]], <!-- ~ -->, <ref ~ />, <ref ~</ref>, <br~>
        txt = re.sub(r'({{(.*?)}})|(\[\[File:(.*?)\n)|(\<\!\-\-(.*?)\-\-\>)|(\<ref(.*?)\/\>)|(\<ref(.*?)\<\/ref\>)|(\<br(\s?\/?)\>)', 
                     '', page['revision']['text']['#text'], 0, re.DOTALL)
        
        # separating internal links
        link = re.findall('(\[\[(.*?)\]\])', txt)
        text_w = re.sub('(\[\[(.*?)\]\])|(\\n)', ' ', txt, 0, re.DOTALL)
        text_s = re.sub('(?<=^\[\[\b).*(?=\b\|(.*?)\]\])|(\\n)',' ',txt, 0, re.DOTALL) #keeping the links first
        text_s = re.sub('(\[\[(.*?)\]\])|(\\n)', ' ', text_s, 0, re.DOTALL)
        
        for c in link:
            if '|' in c[1]:
                sep = c[1].split('|')
                wiki_link = wiki_link + ', ' + sep[0]
                text_w = text_w + ', ' + sep[1]
            else:
                wiki_link = wiki_link + ', ' + c[1]
                text_w = text_w + ', ' + c[1]
                
    df_text = df_text.append({'title': title, 'text': text_w, 'wiki_link': wiki_link, 'redirect': redirect, 'sentences': text_s}, ignore_index=True) 
    

In [129]:
df_text['sentences'][1] # full article text for Anarchism article

'        \'\'\'Anarchism\'\'\' is an     and   that rejects   deemed unjust and advocates their replacement with  ,   societies based on voluntary,   institutions. These institutions are often described as  , although several authors have defined them more specifically as distinct institutions based on non-hierarchical or  . Anarchism\'s central disagreement with other ideologies is that it holds the   to be undesirable, unnecessary, and harmful.  Anarchism is usually placed on the   of the  , and much of its   and   reflect   of  ,  ,  ,  , or  . As anarchism does not offer a fixed body of doctrine from a single particular worldview, many   exist and varieties of anarchy diverge widely.   can differ fundamentally, supporting anything from extreme   to complete  . Strains of anarchism have often been divided into the categories of   and  , or similar dual classifications.  == Etymology, terminology, and definition ==   The etymological origin of the word anarchism is from the ancient G

In [130]:
import nltk
from nltk import download
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adrienawong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [131]:
def preprocess_sentence(doc):
    return tokenize.sent_tokenize(doc)

In [132]:
df_text['sentences'] = df_text['sentences'].apply(preprocess_sentence)

In [133]:
df_text['sentences'][1] #list of sentences for Anarchism article

["        '''Anarchism''' is an     and   that rejects   deemed unjust and advocates their replacement with  ,   societies based on voluntary,   institutions.",
 'These institutions are often described as  , although several authors have defined them more specifically as distinct institutions based on non-hierarchical or  .',
 "Anarchism's central disagreement with other ideologies is that it holds the   to be undesirable, unnecessary, and harmful.",
 'Anarchism is usually placed on the   of the  , and much of its   and   reflect   of  ,  ,  ,  , or  .',
 'As anarchism does not offer a fixed body of doctrine from a single particular worldview, many   exist and varieties of anarchy diverge widely.',
 'can differ fundamentally, supporting anything from extreme   to complete  .',
 'Strains of anarchism have often been divided into the categories of   and  , or similar dual classifications.',
 '== Etymology, terminology, and definition ==   The etymological origin of the word anarchism is 

In [134]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adrienawong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adrienawong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [135]:
def preprocess_word(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    while (doc.count('n')): 
        doc.remove('n') 
    while (doc.count('br')): 
        doc.remove('br') 
    return doc

In [136]:
df_text['text'] = df_text['text'].apply(preprocess_word)

In [137]:
def preprocess_link(doc):
    if doc.startswith(', '):
        doc = doc[2:]
    doc = doc.split(', ')
    return doc

In [138]:
df_text['wiki_link'] = df_text['wiki_link'].apply(preprocess_link)

In [139]:
df_text.head(15)

,title,text,wiki_link,redirect,sentences
0,AccessibleComputing,[],[Computer accessibility],T,[]
1,Anarchism,"[rejects, deemed, unjust, advocates, replaceme...","[Anti-authoritarianism, Political philosophy, ...",F,[ '''Anarchism''' is an and that ...
2,AfghanistanHistory,[],[History of Afghanistan],T,[]
3,AfghanistanGeography,[],[Geography of Afghanistan],T,[]
4,AfghanistanPeople,[],[Demographics of Afghanistan],T,[]
5,AfghanistanCommunications,[],[Communications in Afghanistan],T,[]
6,AfghanistanTransportations,[],[Transport in Afghanistan],T,[]
7,AfghanistanMilitary,[],[Afghan Armed Forces],T,[]
8,AfghanistanTransnationalIssues,[],[Foreign relations of Afghanistan],T,[]
9,AssistiveTechnology,[],[Assistive_technology],T,[]


#### Saving as text file for text modelling (temporary)

In [140]:
#temporarily making text file for anarchism sentences for text modelling code
with open("anarchism_sentences.txt", "w") as output:
    output.write(str(df_text['sentences'][1]))

## Centroid Word Vector representation for each article

In [144]:
from gensim.models import Word2Vec

In [161]:
model = Word2Vec(sentences = df_text['text'], min_count=0, window=2)

In [179]:
model.save('word_emb.model')

In [174]:
def get_centroid_vec(token_list):
    if len(token_list) == 0: return []
    centroid_vec = np.zeros(shape=100)
    for w in token_list:
        vec = model.wv[w]
        centroid_vec = centroid_vec + vec
    return centroid_vec/len(token_list)

In [175]:
df_text['centroid_vec'] =  df_text['text'].apply(get_centroid_vec)

In [176]:
df_text.head()

,title,text,wiki_link,redirect,sentences,centroid_vec
0,AccessibleComputing,[],[Computer accessibility],T,[],[]
1,Anarchism,"[rejects, deemed, unjust, advocates, replaceme...","[Anti-authoritarianism, Political philosophy, ...",F,[ '''Anarchism''' is an and that ...,"[-0.4516527240684108, -0.664244985694551, 0.01..."
2,AfghanistanHistory,[],[History of Afghanistan],T,[],[]
3,AfghanistanGeography,[],[Geography of Afghanistan],T,[],[]
4,AfghanistanPeople,[],[Demographics of Afghanistan],T,[],[]


In [141]:
# I think an excel cell has a limitation to word count. When it's too long, it overflows. You better use df directly instead of csv
df_text.to_csv (r'Results/article_text1.csv', index = False, header=True)

In [142]:
with open('Datasets/pageviews-20200101-000000', encoding='utf8') as file:
    data_view = file.read()

In [143]:
print(data_view[:300])